# excercise_7_1_RAG_basic_example

**Temat:** RAG — chunking → wektory → retrieval → szkic promptu.

## Zadanie 1 — Chunking

In [ ]:
def chunk_text(text: str, max_len: int = 80) -> list[str]:
    words, chunks, cur = text.split(), [], ""
    for w in words:
        if len(cur) + len(w) + 1 <= max_len:
            cur = (cur + " " + w).strip()
        else:
            chunks.append(cur)
            cur = w
    if cur: chunks.append(cur)
    return chunks
txt = "LangChain ułatwia budowę aplikacji opartych o LLM. W RAG łączymy retrieval i generację."
print(chunk_text(txt, 40))


## Zadanie 2 — Proste featury i najbliższe sąsiedztwo

In [ ]:
import numpy as np
def featurize(s: str) -> np.ndarray:
    words = s.split()
    return np.array([len(s), len(set(words))], dtype=float)  # TODO: prosta cecha
def build_index(chunks): return [featurize(c) for c in chunks]
def nearest(query, chunks, index, top_k=2):
    qv = featurize(query)
    dists = [np.linalg.norm(qv - v) for v in index]
    order = np.argsort(dists)[:top_k]
    return [(chunks[i], dists[i]) for i in order]
doc_chunks = chunk_text(txt, 40)
index = build_index(doc_chunks)
print(nearest("retrieval generacja", doc_chunks, index, top_k=2))


## Zadanie 3 — Retrieval → szkic promptu do LLM

In [ ]:
top = [c for c, _ in nearest("Jak łączyć retrieval i generację?", doc_chunks, index, top_k=2)]
prompt = f"Kontekst: {top}\n\nPytanie: Jak łączyć retrieval i generację?"
print(prompt[:200], "...")
assert "Kontekst:" in prompt and "Pytanie:" in prompt
